In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline
best_model = None
best_score = 0
hospital_name = 'host2'

In [2]:
data = pd.read_csv('../data/hospital' + str(2) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (274, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.005
[MICE] Starting imputation round 2/110, elapsed time 0.072
[MICE] Starting imputation round 3/110, elapsed time 0.096
[MICE] Starting imputation round 4/110, elapsed time 0.122
[MICE] Starting imputation round 5/110, elapsed time 0.146
[MICE] Starting imputation round 6/110, elapsed time 0.170
[MICE] Starting imputation round 7/110, elapsed time 0.193
[MICE] Starting imputation round 8/110, elapsed time 0.227
[MICE] Starting imputation round 9/110, elapsed time 0.254
[MICE] Starting imputation round 10/110, elapsed time 0.278
[MICE] Starting imputation round 11/110, elapsed time 0.301
[MICE] Starting imputation round 12/110, elapsed time 0.325
[MICE] Starting imputation round 13/110, elapsed time 0.349
[MICE] Starting imputation round 14/110, elapsed time 0.373
[MICE] Starting imputation round 15/110, elapsed time 0.396
[MICE] Starting imputation round 16/110, elapsed time 0.419
[MI

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'roc_auc'

In [6]:
balancer = SMOTE(random_state=42)

In [7]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [8]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [9]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


In [10]:
gs_dummy.best_params_

{'clf__strategy': 'stratified'}

In [11]:
gs_dummy.best_score_

0.5046998479802258

In [12]:
param_grid = {"clf__max_depth": [10,30,50],
              "clf__criterion": ['gini', 'entropy'],
              "clf__n_estimators": [100,200,500,1000],
              "clf__random_state": [42],
              "clf__class_weight" : [None, 'balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}


rf = RandomForestClassifier()

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', rf))
pipeline = Pipeline(plist)

gs_rf = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [13]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  2.4min finished


In [14]:
gs_rf.best_params_

{'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': 10,
 'clf__n_estimators': 200,
 'clf__random_state': 42}

In [15]:
gs_rf.best_score_


0.599488802037897

In [16]:
if gs_rf.best_score_ > best_score:
    best_model = gs_rf
    best_score = gs_rf.best_score_

In [17]:
show_f_importance(gs_rf.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 45 - Parea_ECG_V8 (0.029903)
2. feature 1 - QTm_old (0.027663)
3. feature 3 - QTn (0.025701)
4. feature 4 - QRS (0.025269)
5. feature 34 - Parea_ECG_I (0.023605)
6. feature 46 - Parea_ECG_V9 (0.022138)
7. feature 37 - Parea_ECG_V1 (0.022081)
8. feature 25 - Pdur_ECG_V4 (0.021937)
9. feature 23 - Pdur_ECG_V3 (0.021894)
10. feature 40 - Parea_ECG_V3R (0.021461)
11. feature 44 - Parea_ECG_V6 (0.019421)
12. feature 0 - RR (0.018803)
13. feature 2 - QR (0.017557)
14. feature 51 - Pfwhm_ECG_II (0.017238)
15. feature 52 - Pfwhm_ECG_III (0.016990)
16. feature 48 - Parea_ECG_aVL (0.016762)
17. feature 5 - PQ (0.016446)
18. feature 12 - Pdur16mean (0.016023)
19. feature 64 - Pfwhm_ECG_aVL (0.015942)
20. feature 39 - Parea_ECG_V3 (0.015654)
21. feature 35 - Parea_ECG_II (0.015572)
22. feature 16 - Pdur16std (0.015502)
23. feature 42 - Parea_ECG_V4R (0.015468)
24. feature 49 - Parea_ECG_aVR (0.015325)
25. feature 53 - Pfwhm_ECG_V1 (0.015190)
26. feature 57 - Pfwhm_ECG_V

In [18]:
param_grid = {"clf__max_depth": [3,5,8],
              "clf__n_estimators": [100,200,500,1000],
              "clf__min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
# 

gb = GradientBoostingClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', gb))
pipeline = Pipeline(plist)
gs_gb = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [19]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.3min finished


In [20]:
gs_gb.best_params_

{'clf__max_depth': 8, 'clf__min_samples_leaf': 3, 'clf__n_estimators': 100}

In [21]:
gs_gb.best_score_

0.5951229754533635

In [22]:
if gs_gb.best_score_ > best_score:
    best_model = gs_gb
    best_score = gs_gb.best_score_

In [23]:
show_f_importance(gs_gb.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 37 - Parea_ECG_V1 (0.048080)
2. feature 2 - QR (0.041954)
3. feature 4 - QRS (0.041024)
4. feature 1 - QTm_old (0.040056)
5. feature 52 - Pfwhm_ECG_III (0.033386)
6. feature 39 - Parea_ECG_V3 (0.030824)
7. feature 45 - Parea_ECG_V8 (0.030709)
8. feature 34 - Parea_ECG_I (0.028410)
9. feature 64 - Pfwhm_ECG_aVL (0.028243)
10. feature 40 - Parea_ECG_V3R (0.027663)
11. feature 54 - Pfwhm_ECG_V2 (0.027552)
12. feature 46 - Parea_ECG_V9 (0.027377)
13. feature 3 - QTn (0.026419)
14. feature 10 - PCarea (0.023709)
15. feature 51 - Pfwhm_ECG_II (0.021092)
16. feature 38 - Parea_ECG_V2 (0.020132)
17. feature 67 - Age (0.019343)
18. feature 0 - RR (0.019154)
19. feature 35 - Parea_ECG_II (0.018251)
20. feature 25 - Pdur_ECG_V4 (0.018163)
21. feature 23 - Pdur_ECG_V3 (0.017383)
22. feature 47 - Parea_ECG_aVF (0.017165)
23. feature 59 - Pfwhm_ECG_V5 (0.016009)
24. feature 48 - Parea_ECG_aVL (0.015237)
25. feature 16 - Pdur16std (0.014984)
26. feature 62 - Pfwhm_ECG_V9 (

In [24]:
param_grid = {"clf__learning_rate": [0.1,0.3,0.7,0.8],
              "clf__n_estimators": [100,1000,2000,2500,3000,5000]}

ab = AdaBoostClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', ab))
pipeline = Pipeline(plist)

gs_ab = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [25]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.8min finished


In [26]:
gs_ab.best_params_

{'clf__learning_rate': 0.8, 'clf__n_estimators': 100}

In [27]:
gs_ab.best_score_

0.5705882827981764

In [28]:

if gs_ab.best_score_ > best_score:
    best_model = gs_ab
    best_score = gs_ab.best_score_

In [29]:
show_f_importance(gs_ab.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 35 - Parea_ECG_II (0.070000)
2. feature 34 - Parea_ECG_I (0.050000)
3. feature 10 - PCarea (0.050000)
4. feature 64 - Pfwhm_ECG_aVL (0.050000)
5. feature 40 - Parea_ECG_V3R (0.050000)
6. feature 45 - Parea_ECG_V8 (0.050000)
7. feature 37 - Parea_ECG_V1 (0.040000)
8. feature 62 - Pfwhm_ECG_V9 (0.040000)
9. feature 59 - Pfwhm_ECG_V5 (0.040000)
10. feature 39 - Parea_ECG_V3 (0.030000)
11. feature 25 - Pdur_ECG_V4 (0.030000)
12. feature 50 - Pfwhm_ECG_I (0.030000)
13. feature 1 - QTm_old (0.030000)
14. feature 2 - QR (0.030000)
15. feature 61 - Pfwhm_ECG_V8 (0.030000)
16. feature 55 - Pfwhm_ECG_V3 (0.020000)
17. feature 54 - Pfwhm_ECG_V2 (0.020000)
18. feature 32 - Pdur_ECG_aVL (0.020000)
19. feature 30 - Pdur_ECG_V9 (0.020000)
20. feature 3 - QTn (0.020000)
21. feature 42 - Parea_ECG_V4R (0.020000)
22. feature 44 - Parea_ECG_V6 (0.020000)
23. feature 57 - Pfwhm_ECG_V4 (0.020000)
24. feature 47 - Parea_ECG_aVF (0.020000)
25. feature 23 - Pdur_ECG_V3 (0.020000)
2

In [30]:
param_grid = {"clf__max_depth": [3,5,8,10,15,30],
              "clf__n_estimators": [100,200,500,1000,1500],
              "clf__min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)

plist = []
plist.append(('balancer', balancer))
plist.append(('clf', et))
pipeline = Pipeline(plist)

gs_et = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [31]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.6min finished


In [32]:
gs_et.best_params_

{'clf__max_depth': 3, 'clf__min_samples_leaf': 10, 'clf__n_estimators': 500}

In [33]:
gs_et.best_score_

0.6312963660820206

In [34]:
if gs_et.best_score_ > best_score:
    best_model = gs_et
    best_score = gs_et.best_score_

In [35]:
show_f_importance(gs_et.best_estimator_.named_steps['clf'],x_df)

Feature ranking:
1. feature 1 - QTm_old (0.047928)
2. feature 23 - Pdur_ECG_V3 (0.040189)
3. feature 22 - Pdur_ECG_V2 (0.032469)
4. feature 25 - Pdur_ECG_V4 (0.030794)
5. feature 45 - Parea_ECG_V8 (0.030459)
6. feature 12 - Pdur16mean (0.030421)
7. feature 3 - QTn (0.029327)
8. feature 28 - Pdur_ECG_V6 (0.029277)
9. feature 21 - Pdur_ECG_V1 (0.029096)
10. feature 8 - PCdur (0.025332)
11. feature 40 - Parea_ECG_V3R (0.024362)
12. feature 13 - Pdur16median (0.024003)
13. feature 33 - Pdur_ECG_aVR (0.023216)
14. feature 15 - Pdur16max (0.023134)
15. feature 11 - P_LoffEon (0.023072)
16. feature 31 - Pdur_ECG_aVF (0.021479)
17. feature 0 - RR (0.021276)
18. feature 65 - Pfwhm_ECG_aVR (0.020543)
19. feature 60 - Pfwhm_ECG_V6 (0.020306)
20. feature 18 - Pdur_ECG_I (0.020173)
21. feature 32 - Pdur_ECG_aVL (0.020143)
22. feature 59 - Pfwhm_ECG_V5 (0.019319)
23. feature 14 - Pdur16min (0.018990)
24. feature 24 - Pdur_ECG_V3R (0.017689)
25. feature 37 - Parea_ECG_V1 (0.017261)
26. feature 27 - P

In [36]:
cdata = pd.read_csv('../data/central' +'_data.csv')
cdata.shape

(275, 73)

In [37]:
data_e = cdata[cdata['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
# df_columns = data.columns.tolist()
# matching_columns1 = [s for s in df_columns if "Parea" in s]
# matching_columns2 = [s for s in df_columns if "Pdur" in s]
# mc = matching_columns1 + matching_columns2
# data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
cy_df = data_e['AFclass'].copy()
cx_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (270, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.004
[MICE] Starting imputation round 2/110, elapsed time 0.030
[MICE] Starting imputation round 3/110, elapsed time 0.057
[MICE] Starting imputation round 4/110, elapsed time 0.098
[MICE] Starting imputation round 5/110, elapsed time 0.122
[MICE] Starting imputation round 6/110, elapsed time 0.149
[MICE] Starting imputation round 7/110, elapsed time 0.176
[MICE] Starting imputation round 8/110, elapsed time 0.201
[MICE] Starting imputation round 9/110, elapsed time 0.231
[MICE] Starting imputation round 10/110, elapsed time 0.256
[MICE] Starting imputation round 11/110, elapsed time 0.287
[MICE] Starting imputation round 12/110, elapsed time 0.311
[MICE] Starting imputation round 13/110, elapsed time 0.334
[MICE] Starting imputation round 14/110, elapsed time 0.360
[MICE] Starting imputation round 15/110, elapsed time 0.392
[MICE] Starting imputation round 16/110, elapsed time 0.418
[MI

In [43]:
from sklearn.metrics import roc_auc_score
p = best_model.predict(cx_df);
scores = pd.DataFrame({hospital_name: p})
scores.to_csv('../data/' + hospital_name +'_predictions.csv', index=False)
np.save('../data/' + hospital_name +'_acc',best_score)
roc_auc_score(cy_df, p)

0.48360655737704916

In [42]:
scores.head(5)

,host2
0,1.0
1,1.0
2,0.0
3,1.0
4,0.0


In [44]:
best_model

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('balancer', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=42, ratio='auto', svm_estimator=None)), ('clf', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', ...estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [3, 5, 8, 10, 15, 30], 'clf__n_estimators': [100, 200, 500, 1000, 1500], 'clf__min_samples_leaf': [1, 3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [40]:
best_score

0.6312963660820206

In [39]:
from sklearn.externals import joblib
joblib.dump(best_model.best_estimator_, '../data/' + hospital_name + '_model.pkl')

['../data/host2_model.pkl']